# Cost Estimation Analysis of 3D Printed Parts

<center>
    <img src="img/01.jpg" alt="Alt text" width="300">
</center>

Calculating the cost of a 3D printed part involves a multifaceted approach that considers several key factors. Firstly, labor plays a crucial role, encompassing the time spent on design, preparation, and post-processing. Printer cost, including depreciation, maintenance, and energy consumption, adds to the overall expense. The choice of material is pivotal, as different types vary widely in price and suitability for the application. Moreover, the volume and shape of the part influence material usage and print time, directly impacting costs. Each of these elements must be meticulously evaluated to accurately determine the total cost of producing a 3D printed part, ensuring economic feasibility and quality adherence.

#### Press ▶️ to select a GCode File

In [ ]:
# @title Import Packages

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import interactive
import re
from functools import partial
%matplotlib inline

# @title Class Definition

# GcodeFileHandler

## Display "Upload GCODE" button
## When "Upload GCODE" is pressed; stores uploaded gcode files in dictionary gcode_storage --> {key: Filename, value: file contents}
class GCodeFileHandler:
    def __init__(self):
        self.file_select = widgets.Dropdown(
            options=self.get_gcode_files(),
            description='Select GCODE'
        )
        self.load_button = widgets.Button(description="Load File")
        self.reset_button = widgets.Button(description="Reset")
        self.gcode_storage = {}

        self.load_button.on_click(self.on_load_button_clicked)
        self.reset_button.on_click(self.on_reset_button_clicked)

        display(self.file_select)
        display(self.load_button)
        display(self.reset_button)

    def get_gcode_files(self):
        gcode_dir = 'gcode'
        return [(file, os.path.join(gcode_dir, file)) for file in os.listdir(gcode_dir) if file.endswith('.gcode')]

    def on_load_button_clicked(self, b):
        if self.file_select.value:
            file_path = self.file_select.value
            with open(file_path, 'r') as file:
                content = file.read()
            filename = os.path.basename(file_path)
            self.gcode_storage[filename] = content
            print(f"Loaded: {filename}")
        else:
            print("No file selected.")

    def on_reset_button_clicked(self, b):
        self.gcode_storage.clear()

        clear_output(wait=True)

        self.file_select.options = self.get_gcode_files()

        display(self.file_select)
        display(self.load_button)
        display(self.reset_button)
        print("Reset successful.")

# GcodeProcessor

## Display "Calculate Cost" button in corresponding cell
## When "Calculate Cost" is pressed; use dictionary to create an instance of GcodeProcessor for each key-value pair in gcode_storage; each instance takes the key and value of one element as input,
### storing the output in dictionary, filename_factors, --> {key: name of data, value: data}

class GcodeProcessor:
    def __init__(self, gcode_content):
        self.gcode = gcode_content

    def extract_layers(self):
        num_layers = self.gcode.count(';LAYER_CHANGE')

        return num_layers

    def extract_print_time(self):
        # Regular expression to match the estimated print time from gcode
        pattern = r"estimated printing time \(normal mode\) = (?:(\d+h )?(\d+m )?(\d+s)?)"
        match = re.search(pattern, self.gcode)
        
        if match:
            # Extract hours, minutes, and seconds from the matched groups
            hours = int((match.group(1) or '0h').strip('h '))
            minutes = int((match.group(2) or '0m').strip('m '))
            seconds = int((match.group(3) or '0s').strip('s '))

            # Calculate total hours
            total_seconds = hours * 3600 + minutes * 60 + seconds
            total_hours = total_seconds / 3600

            # Construct print time string
            print_time_str = f"{hours}h {minutes}m {seconds}s"

            return total_hours, print_time_str
        else:
            return "Print time not found", None

    def extract_filament_type(self):
        # Regular expression to match the filament type
        pattern = r"filament_type = (\w+)"
        match = re.search(pattern, self.gcode)

        if match:
            return match.group(1)
        else:
            return "Filament type not found"
    
    def extract_filament_usage_g(self):
        # Regular expression to match the filament usage in grams
        pattern = r"; filament used \[g\] = ([\d.]+)"
        match = re.search(pattern, self.gcode)

        if match:
            return float(match.group(1))
        else:
            return "Filament usage (g) not found"
    
    def extract_filament_usage_cm3(self):
        # Regular expression to match the filament usage in cubic centimeters
        pattern = r"; filament used \[cm3\] = ([\d.]+)"
        match = re.search(pattern, self.gcode)

        if match:
            return float(match.group(1))
        else:
            return "Filament usage (cm3) not found"
    
# CostCalculator

## Takes the data pulled from the Gcode in GcodeProcessor as input and returns the dataframe, cost_df, containing the types of cost and their values
## Displays cost dataframe


class CostCalculator:
    def __init__(self, print_hrs, fil_used_cm3):
        # define default values for cost calculation
        self.fil_cost = 0.05
        self.labor_rate = 20
        self.kwh_cost = 0.203
        self.machine_cost = 3000

        # initiate input params
        self.print_hrs = print_hrs
        self.fil_used_cm3 = fil_used_cm3

    def costFunc(self):

        # Calculate costs
        maint_cost_per_hr = (self.machine_cost / 43800) # 5 years in hours
        # total material cost
        filament_cost = self.fil_used_cm3 * self.fil_cost
        # toal labor cost
        labor_cost = (self.print_hrs/20) * self.labor_rate
        # total electricity cost
        elect_cost = self.kwh_cost * self.print_hrs
        # total depreciation from machine cost
        maint_this_print = maint_cost_per_hr * self.print_hrs

        total_cost = filament_cost + labor_cost + elect_cost + maint_this_print

        # Create DataFrame for cost breakdown
        data = {
            'Factor': ['Material', 'Labor', 'Electricty', 'Maintainence', 'Total'],
            'Cost ($)': [filament_cost, labor_cost, elect_cost, maint_this_print, total_cost]
        }
        cost_df = pd.DataFrame(data)
        return cost_df


# GcodeFileSelecter

## Creates a dropdown menu from which the user can select a gcodefile to explore cost factors with
class GCodeFileSelector:
    def __init__(self, gcode_storage):
        self.gcode_storage = gcode_storage # dict
        self.selected_file = None # returned from self.dropdown.value str
        self.print_hrs = None # float
        self.fil_used_cm3 = None # float
        self.menu = self.gcode_storage.keys() # all keys from input dict
        self.gcode_selected = None

        self.dropdown = widgets.Dropdown(
            options=self.menu,
            # options=self.gcode_storage.keys(),
            value=None,
            description='GCODE Files:'
        )

        self.confirm_button = widgets.Button(description='Confirm Selection')

        self.confirm_button.on_click(self.on_confirm_button_clicked)

        display(self.dropdown)
        display(self.confirm_button)



    def on_confirm_button_clicked(self, b):
        if self.dropdown.value:
            self.selected_file = self.dropdown.value
            print(f"Confirmed file: {self.selected_file}")


            self.gcode_selected = self.gcode_storage[self.selected_file]
            self.process_gcode_instance = GcodeProcessor(self.gcode_selected)

            self.print_hrs, self.print_str = self.process_gcode_instance.extract_print_time()
            self.fil_used_cm3 = self.process_gcode_instance.extract_filament_usage_cm3()


        else:
            print("No file selected.")

    def get_selected_file(self):
        return self.selected_file, self.print_hrs, self.fil_used_cm3


# CostExplorer
## Prompt user to select file from gcode_storage from dropdown menu
## Display sliders for certain cost parameters, with default settings matching those used in CostCalculator
## Display live pie chart with cost factors as the slices for visualization

class CostExploreWidgets:
    def __init__(self, print_hrs, fil_used_cm3):
        self.print_hrs = print_hrs
        self.fil_used_cm3 = fil_used_cm3
        self.total_cost = 0
        self.slices = {}
        self.outputChart = widgets.Output()  # Dedicated Output widget for plotting

    def gen_widgets(self):
        self.fil_cost_slider = widgets.FloatSlider(value=0.05, min=0, max=1, step=0.01, description='Filament Cost per cm3:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})
        self.labor_rate_slider = widgets.FloatSlider(value=20, min=0, max=50, step=1, description='Labor Cost per Hour:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})
        self.kwh_cost_slider = widgets.FloatSlider(value=0.203, min=0, max=1, step=0.001, description='Cost per kWh:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})
        self.machine_cost_slider = widgets.FloatSlider(value=3000, min=0, max=10000, step=100, description='Cost of 3D Printer:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})
        self.print_hrs_slider = widgets.FloatSlider(value = self.print_hrs, min=0, max=(self.print_hrs*10), step=0.05, description='Time to Print in Hours:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})
        self.fil_used_slider = widgets.FloatSlider(value=self.fil_used_cm3, min=0, max=(self.fil_used_cm3*10), step=1, description='Filament Used in cm3:', layout=widgets.Layout(width='600px'), style={'description_width': '150px'})

        # Clear and display the sliders within the output widget
        with self.outputChart:
            clear_output(wait=True)
            display(self.fil_cost_slider, self.labor_rate_slider, self.kwh_cost_slider, self.machine_cost_slider, self.print_hrs_slider, self.fil_used_slider)

    def calc_slices(self):
        fil_cost = self.fil_cost_slider.value
        labor_rate = self.labor_rate_slider.value
        kwh_cost = self.kwh_cost_slider.value
        machine_cost = self.machine_cost_slider.value
        print_hrs = self.print_hrs_slider.value
        fil_used = self.fil_used_slider.value

        maint_cost_per_hr = (machine_cost / 43800) # 5 years in hours
        material_cost = fil_used * fil_cost
        labor_cost = (print_hrs / 20) * labor_rate
        electricity_cost = kwh_cost * print_hrs
        maintenance_cost = maint_cost_per_hr * print_hrs

        self.slices = {
            'Material': material_cost,
            'Labor': labor_cost,
            'Electricity': electricity_cost,
            'Maintenance': maintenance_cost
        }
        self.total_cost = sum(self.slices.values())

    def plot_chart(self):
        self.calc_slices()
        with self.outputChart:
            clear_output(wait=True)
            fig, ax = plt.subplots()

            labels = list(self.slices.keys())
            sizes = list(self.slices.values())
            explode = (0.1, 0, 0, 0)  # Example explode for the first slice

            pie_chart = ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=False, startangle=140)
            ax.axis('equal')
            ax.set_title('Cost Factors')

            total_str = f'Total: ${self.total_cost:.2f}'
            ax.text(0, -1.2, total_str, fontsize=12, ha='center')

            plt.show()

class CostController:
    def __init__(self, print_hrs, fil_used_cm3):
        self.explore = CostExploreWidgets(print_hrs, fil_used_cm3)
        self.output = widgets.Output()

    def run(self):
        with self.output:
            self.explore.gen_widgets()

        gen_button = widgets.Button(description="Generate Chart", layout=widgets.Layout(width='800px'))
        reset_button = widgets.Button(description="Restore Defaults", layout=widgets.Layout(width='800px'))

        def gen_chart(b):
            with self.output:
                clear_output(wait=True)
                self.explore.plot_chart()

        def reset_sliders(b):
            with self.output:
                clear_output(wait=True)
                self.explore.gen_widgets()

        gen_button.on_click(gen_chart)
        reset_button.on_click(reset_sliders)

        display(gen_button, reset_button, self.explore.outputChart, self.output)


# @title Upload Gcode File(s)

# Instance of GcodeFileHandler
import os
gcode_handler = GCodeFileHandler()

The G-code file used in 3D printing encapsulates all essential print parameters beyond mere movement instructions. It includes crucial details such as material specifications, print time estimates, cost calculations based on material usage and energy consumption, as well as other pertinent settings essential for the accurate reproduction of the intended model. This comprehensive data within the G-code ensures precise execution and provides valuable insights into the production process, contributing significantly to both efficiency and cost-effectiveness in 3D printing operations.

#### Press ▶️ to pull cost and print information from GCode

In [ ]:
# Assuming you've already imported widgets
from IPython.display import display

# @title Gcode Details
get_details_b = widgets.Button(description='Get Print Details')
display(get_details_b)

details_output = widgets.Output()  # Output widget to display results

def process_instances(b):
    with details_output:  # Use the output widget to display results
        details_output.clear_output()  # Clear previous outputs
        for filename, content in gcode_handler.gcode_storage.items():
            processor = GcodeProcessor(content)

            num_layers = processor.extract_layers()
            print_hrs, print_time, *_ = processor.extract_print_time()
            fil_type = processor.extract_filament_type()
            fil_used_g = processor.extract_filament_usage_g()
            fil_used_cm3 = processor.extract_filament_usage_cm3()

            print(f"Print Details for {filename}:")
            print(f"Number of layers: {num_layers}")
            print(f"Print Time (hrs): {print_time}")
            print(f"Filament Type: {fil_type}")
            print(f"Filament Used (g): {fil_used_g}")
            print(f"Filament Used (cm3): {fil_used_cm3}")
            print(" ")

get_details_b.on_click(process_instances)
display(details_output)  # Display the output widget below the button


#### Press ▶️ to calculate the costs

Calculating costs in 3D printing, factoring in printer depreciation based on print time, involves a methodical approach. Depreciation accounts for the wear and tear of the printer over its lifespan, influenced primarily by the cumulative hours of operation. By dividing the total depreciation cost of the printer by its expected operational hours, a per-hour depreciation rate can be determined. This rate is then multiplied by the print time of each job to allocate a portion of the printer's overall depreciation to that specific print. This method not only spreads out equipment costs more accurately across projects but also aids in pricing decisions, ensuring that each print job reflects a fair share of the printer's wear and tear over time.

In [ ]:
# Assuming you've already imported widgets and IPython display function
import ipywidgets as widgets
from IPython.display import display

# @title Calculate Costs
get_cost = widgets.Button(description='Calculate Costs')
display(get_cost)

cost_compare = {}
cost_output = widgets.Output()  # Output widget to display results

def cost_instances(b):
    global cost_compare
    with cost_output:  # Use the output widget to display results
        cost_output.clear_output()  # Clear previous outputs
        for filename, content in gcode_handler.gcode_storage.items():
            processor = GcodeProcessor(content)

            print_hrs, print_time = processor.extract_print_time()
            fil_used_cm3 = processor.extract_filament_usage_cm3()

            cost_calc = CostCalculator(print_hrs, fil_used_cm3)
            cost_df = cost_calc.costFunc()

            print(f'Filename: {filename}')
            display(cost_df)  # Display the DataFrame in the output widget

            cost_compare[filename] = cost_df
            print(" ")

get_cost.on_click(cost_instances)
display(cost_output)  # Display the output widget below the button


#### Press ▶️ to confirm the GCode of interest

In [ ]:
# @title Select Gcode File to Explore Cost Factors
file_selector = GCodeFileSelector(gcode_handler.gcode_storage)

#### Press ▶️ to do a cost estimation and analysis based on various pricing parameters

In [ ]:
# @title Activate the Cost Explorer

selected_file, print_hrs, fil_used_cm3 = file_selector.get_selected_file()
controls = CostController(print_hrs, fil_used_cm3)
controls.run()